In [43]:
import pandas as pd
import h5py
import os
import h5py
import os
from scipy.stats import skew, kurtosis
import scipy.fftpack
import scipy.signal as signal


from sklearn.preprocessing import LabelEncoder
from scipy.stats import skew, kurtosis
from tensorflow.keras.utils import to_categorical

def zero_crossing_rate(arr):
    """
    Sinyalde işaret değişimlerinin sayısını normalize ederek döndürür.
    """
    return np.sum(np.diff(np.sign(arr)) != 0) / len(arr)

In [44]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
import cv2
import numpy as np

# Mobilenet modeli sadece feature çıkarımı için
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
feature_model = Model(inputs=base_model.input, outputs=GlobalAveragePooling2D()(base_model.output))

/var/folders/kw/3t8qd9fd6hlfbqdps6pvpchw0000gn/T/ipykernel_60414/841802844.py:8: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(64, 64, 3))


In [ ]:
X = []
y = []


for file in os.listdir("gestureData/"):
    if file.endswith(".mat"):
        user_id = int(file.split("_")[2])
        with h5py.File(f"gestureData/{file}", "r") as f:
            key = list(f.keys())[0]  # genellikle 'depth_sequence'
            sequence = f[key][:]  # [frames, height, width]
            
            # Sadece ilk 10 kare, her kareyi 64x64'e yeniden boyutlandır
            if sequence.shape[0] < 10:
                continue
            
            # frames = sequence[:10]
            # resized = [cv2.resize(frame, (64, 64)) for frame in frames]
            # stacked = np.stack(resized, axis=0)  # [25, 64, 64]

            frame_features = []
            for frame in sequence:
                resized = cv2.resize(frame, (64, 64))
                if len(resized.shape) == 2:
                    resized = np.stack([resized]*3, axis=-1)  # grayscale → RGB
                resized = resized.astype(np.float32) / 255.0
                feat = feature_model.predict(resized[np.newaxis, ...], verbose=0)
                frame_features.append(feat[0])
            
            frame_features = np.array(frame_features)  # [num_frames, 1280]
            mean = np.mean(frame_features, axis=0)
            std = np.std(frame_features, axis=0)
            median = np.median(frame_features, axis=0)
            skew_values = skew(frame_features, axis=0)
            kurtosis_value = kurtosis(frame_features, axis=0)

            
            fft_value = np.abs(scipy.fftpack.fft(frame_features))  # Yalnızca pozitif frekanslar
            fft_value = fft_value / len(frame_features)  # Normalizasyon
            fft_mean = np.mean(fft_value, axis=0)
            fft_std  = np.std(fft_value, axis=0)
            fft_median  = np.median(fft_value, axis=0)
            fft_skew_values = skew(fft_value, axis=0)
            fft_kurtosis = kurtosis(fft_value, axis=0)


            concatenated = np.concatenate([mean, std, median, skew_values, kurtosis_value, fft_mean, fft_std, fft_median, fft_skew_values, fft_kurtosis])  # [2560]

            X.append(concatenated)
            y.append(user_id)


TypeError: 'numpy.ndarray' object is not callable

In [ ]:
X = np.array(X)  # shape: 
y = np.array(y)

# CNN için channel ekle (örneğin grayscale için kanal sayısı = 1)
X = X[..., np.newaxis]  # 
X = X.reshape(333, X.shape[1])

In [32]:
len(y)

333

In [33]:
# y zaten elinde numpy array olarak var
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
num_classes = len(np.unique(y_encoded))  # 10 sınıf
y_cat = to_categorical(y_encoded, num_classes=num_classes)

In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


# Eğitim/test böl
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)

# Model
model = Sequential([
    Dense(512, activation='relu', input_shape=(X.shape[1],)),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()
model.fit(X_train, y_train, epochs=35, batch_size=16, validation_data=(X_test, y_test))


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 512)            │     3,932,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,066,570 (15.51 MB)

 Trainable params: 4,066,570 (15.51 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/35
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.1171 - loss: 4.4225 - val_accuracy: 0.0896 - val_loss: 2.4153
Epoch 2/35
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.1378 - loss: 2.9365 - val_accuracy: 0.1940 - val_loss: 2.2313
Epoch 3/35
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.1511 - loss: 2.3857 - val_accuracy: 0.1642 - val_loss: 2.2404
Epoch 4/35
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1750 - loss: 2.3390 - val_accuracy: 0.1642 - val_loss: 2.1716
Epoch 5/35
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.2099 - loss: 2.1091 - val_accuracy: 0.1791 - val_loss: 2.2076
Epoch 6/35
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.2206 - loss: 2.0996 - val_accuracy: 0.2090 - val_loss: 2.1114
Epoch 7/35
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1677 - loss: 2.1284 - val_accuracy: 0.1791 - val_loss: 2.1433
Epoch 8/35
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.2668 - loss: 2.0196 - val_accuracy: 0.2090 - v

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train_simple, y_test_simple = train_test_split(X, y, test_size=0.2, random_state=42)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train_simple)

y_pred = clf.predict(X_test)
print(classification_report(y_test_simple, y_pred))

# Accuracy
acc = accuracy_score(y_test_simple, y_pred)
print(f"✅ Accuracy: {acc:.4f}")

              precision    recall  f1-score   support

           1       0.40      0.50      0.44         4
           2       0.67      0.40      0.50         5
           3       0.25      0.20      0.22         5
           4       0.50      0.83      0.62         6
           5       0.60      0.75      0.67         8
           6       0.33      0.33      0.33         3
           7       0.50      0.45      0.48        11
           8       0.33      0.11      0.17         9
           9       0.64      1.00      0.78         7
          10       0.88      0.78      0.82         9

    accuracy                           0.55        67
   macro avg       0.51      0.54      0.50        67
weighted avg       0.53      0.55      0.52        67

✅ Accuracy: 0.5522
